In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from matplotlib import pyplot as plt
import os
cwd = os.getcwd()

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

test_filepath = '/kaggle/input/UjiIndoorLoc/ValidationData.csv'
train_filepath = '/kaggle/input/UjiIndoorLoc/TrainingData.csv'
# Any results you write to the current directory are saved as output.

In [ ]:
training_data = pd.read_csv(train_filepath)
test_data = pd.read_csv(test_filepath)
train_lat_long_X = training_data.drop(columns=["LONGITUDE", "LATITUDE", "FLOOR", "BUILDINGID", "SPACEID", "RELATIVEPOSITION", "USERID", "PHONEID", "TIMESTAMP"])
train_lat_long_Y = training_data.loc[:, "LONGITUDE":"LATITUDE"]
test_lat_long_X = test_data.drop(columns=["LONGITUDE", "LATITUDE", "FLOOR", "BUILDINGID", "SPACEID", "RELATIVEPOSITION", "USERID", "PHONEID", "TIMESTAMP"])
test_lat_long_Y = test_data.loc[:, "LONGITUDE":"LATITUDE"]

In [ ]:
def get_accuracy(predictions, actual_values):
  correct_classif = [1 if pred_i == actual_i else 0 for pred_i, actual_i in zip(predictions, actual_values)]
  correct_classif = np.array(correct_classif)
  return correct_classif.sum() / len(actual_values)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(2, activation = 'linear')
])

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
val_data = (test_lat_long_X, test_lat_long_Y)

In [ ]:
epochs = 200
result = model.fit(train_lat_long_X.values.astype(float), train_lat_long_Y.values.astype(float), validation_data = val_data, epochs=epochs)

In [ ]:
x_vals = range(60, epochs+1)
plt.plot(x_vals, result.history['loss'][5 9:], label = 'Train Loss')
#plt.plot(x_vals, result.history['val_loss'], label = 'Validation Loss')
plt.legend()
plt.show()

In [ ]:
x_vals = range(1, epochs+1)
plt.plot(x_vals, result.history['val_loss'], label = 'Validation Loss')
plt.legend()
plt.show()

From the graphs shown above, it can be seen that epochs = 25 is a better value.****

In [ ]:
result = model.fit(train_lat_long_X.values.astype(float), train_lat_long_Y.values.astype(float), validation_data = val_data, epochs=25)

In [ ]:
predictions = model.predict(test_lat_long_X)

In [ ]:
predictions = pd.DataFrame({'LONGITUDE': predictions[:, 0], 'LATITUDE': predictions[:, 1]})

In [ ]:
predictions - test_lat_long_Y